> ## https://www.goratings.org

In [7]:
from bs4 import BeautifulSoup
import requests

import pandas as pd

In [8]:
def get_players_info(input_data):
    name = input_data.split(sep='\n')[1].split(sep='title')[1][1:-2]
    df = pd.read_html(page.text)[0]
    df = df[(df[0] != 'Link')]
    new_row = pd.DataFrame({0:'Name', 1:name}, index =[0])
    df = pd.concat([new_row, df]).transpose()
    df.columns = df.iloc[0]
    df = df[df.Name != 'Name']
    return df

In [9]:
def get_parties(input_data):    
    name = input_data.split(sep='\n')[1].split(sep='title')[1][1:-2]
    input_data = input_data.replace('>â\x99\x82<', '>man<')
    input_data = input_data.replace('>â\x99\x80<', '>woman<')
    input_data = input_data.replace('<img src="/flags/', '')
    input_data = input_data.replace('<a href="http://www.go4go.net/go/games/sgfview/', 
                                    'http://www.go4go.net/go/games/sgfview/')
    input_data = input_data.replace('">View game<', '<')

    df = pd.read_html(input_data)[1]
    df.insert(0, "Name", name, True)
    df['Opponent.3'] = df['Opponent.3'].apply(lambda x: x.split(sep='.')[0])
    df = df.rename(columns={"Opponent.1": "Opponent_Rating", 
                            "Opponent.2": "Opponent_Sex",
                            "Opponent.3": "Opponent_Flag"})
    
    df['id_game'] = df['Kifu'].apply(lambda x: x.split(sep='/')[-1])
    
    return df

In [10]:
frames_players = []
frames_parties = []

print('Pages processed:')

for i in range(1, 2501):
    if (i%100 == 0):
        print(f'\t{i}')
    url = f'https://www.goratings.org/en/players/{i}.html'
    page = requests.get(url)
    raw_data = page.text
    try:
        frames_players.append(get_players_info(raw_data))
    except:
        pass
    try:
        frames_parties.append(get_parties(raw_data))
    except:
        pass
    
df_players = pd.concat(frames_players).reset_index()
df_players = df_players.drop(columns='index')
    
df_parties = pd.concat(frames_parties).reset_index()
df_parties = df_parties.drop(columns='index')

Pages processed:
	100
	200
	300
	400
	500
	600
	700
	800
	900
	1000
	1100
	1200
	1300
	1400
	1500
	1600
	1700
	1800
	1900
	2000
	2100
	2200
	2300
	2400
	2500


In [34]:
df_players = df_players[~df_players['Name'].str.contains('Dummy')]
df_players

,Name,Wins,Losses,Total,Date of Birth
0,Ando Takeo,1,1,2,1938-09-11
1,Li Chunhua,38,47,85,1973-04-18
2,Li Liang,19,22,41,1971-07-04
3,Li Daichun,11,3,14,1977-12-12
4,Lee Sedol,916,456,1372,1983-03-02
...,...,...,...,...,...
2455,Eom Donggun,0,1,1,2000-06-09
2456,Lee Seoyoung,2,6,8,2005-08-12
2457,Hamakado Hibiki,0,1,1,2001-10-04
2458,Huang Qiuxuan,0,1,1,-


In [35]:
df_parties

,Name,Date,Rating,Color,Result,Opponent,Opponent_Rating,Opponent_Sex,Opponent_Flag,Kifu,id_game
0,Ando Takeo,1976-03-03,2987,White,Loss,Miyazawa Goro,3075,man,jp,http://www.go4go.net/go/games/sgfview/80700,80700
1,Ando Takeo,1976-01-14,2988,White,Win,Kodama Kunio,2813,man,jp,http://www.go4go.net/go/games/sgfview/80685,80685
2,Li Chunhua,2016-09-22,2823,Black,Loss,Zhao Yifang,2961,woman,cn,http://www.go4go.net/go/games/sgfview/57892,57892
3,Li Chunhua,2016-09-13,2823,Black,Loss,Pan Yang,3012,woman,cn,http://www.go4go.net/go/games/sgfview/57320,57320
4,Li Chunhua,2016-09-03,2824,White,Loss,Zhang Zihan,3041,woman,cn,http://www.go4go.net/go/games/sgfview/56956,56956
...,...,...,...,...,...,...,...,...,...,...,...
218686,Lee Seoyoung,2023-09-03,2811,Black,Win,Kim Sangin,2876,woman,kr,http://www.go4go.net/go/games/sgfview/110907,110907
218687,Hamakado Hibiki,2023-09-18,2894,White,Loss,Kuwabara Ki,2983,man,jp,http://www.go4go.net/go/games/sgfview/111207,111207
218688,Huang Qiuxuan,2023-09-24,2970,White,Loss,Hu Zihao,3361,man,cn,http://www.go4go.net/go/games/sgfview/111290,111290
218689,Kim Minji,2023-10-16,3041,Black,Loss,Choi Jeong,3430,woman,kr,http://www.go4go.net/go/games/sgfview/111649,111649


In [36]:
player_info = dict()

for i in df_parties['Name'].unique():
    df_temp = df_parties[(df_parties['Name'] == i)]
    df_temp = df_temp[(df_temp['Date'] == df_temp['Date'].max())]

    player_info[i] = list(df_temp['Rating'])
    
for i in df_parties['Opponent'].unique():
    if i not in player_info:
        df_temp = df_parties[(df_parties['Opponent'] == i)]
        df_temp = df_temp[(df_temp['Date'] == df_temp['Date'].max())]
        player_info[i] = list(df_temp['Opponent_Rating'])

In [37]:
for i in df_parties['Opponent'].unique():
    df_temp = df_parties[(df_parties['Opponent'] == i)].head(1)  
    player_info[i].extend([list(df_temp['Opponent_Sex'])[0], list(df_temp['Opponent_Flag'])[0]])

In [38]:
player_info['Kobayashi Koichi']

[3025, 'man', 'jp']

In [39]:
for i in player_info:
    if len(player_info[i])>3:
        player_info[i] = player_info[i][-3:]

In [40]:
df_player_additional_info = pd.DataFrame.from_dict(player_info, orient='index')
df_player_additional_info = df_player_additional_info.reset_index()

df_player_additional_info.columns = ['Name', 'Rating', 'Sex', 'Flag']

In [41]:
df_player_additional_info

,Name,Rating,Sex,Flag
0,Ando Takeo,2987,man,jp
1,Li Chunhua,2823,woman,cn
2,Li Liang,3052,man,cn
3,Li Daichun,3304,man,cn
4,Lee Sedol,3444,man,kr
...,...,...,...,...
2266,Kim Minji,3041,woman,kr
2267,Kobayashi Koichi,3025,man,jp
2268,Kwon Hyojin (m),3196,man,kr
2269,Jeong Woojin,3042,man,kr


In [42]:
df_player_info = df_player_additional_info.merge(df_players, on = 'Name')
df_player_info = df_player_info.sort_values(by=['Rating'], ascending=False)

In [43]:
df_player_info

,Name,Rating,Sex,Flag,Wins,Losses,Total,Date of Birth
1221,Shin Jinseo,3866,man,kr,657,190,847,2000-03-17
998,Park Junghwan,3691,man,kr,893,365,1258,1993-01-11
1650,Wang Xinghao,3678,man,cn,156,69,225,2004-02-02
1161,Gu Zihao,3678,man,cn,314,192,506,1998-03-13
1103,Ke Jie,3666,man,cn,575,247,822,1997-08-02
...,...,...,...,...,...,...,...,...
1174,Tanimiya Ayako,2371,woman,jp,0,15,15,1946-02-02
658,Tafu Kae,2367,woman,jp,1,13,14,1971-03-13
301,Han Cheolkyun,2358,man,kr,1,31,32,1955-01-04
648,Hashiguchi Mika,2290,woman,jp,0,10,10,1963-09-19


In [44]:
df_parties.to_csv("df_parties.csv", index=False)
df_player_info.to_csv("df_players.csv", index=False)

## https://www.go4go.net/

In [1]:
from bs4 import BeautifulSoup
import requests

import pandas as pd

In [2]:
frames_tournaments = []

for i in range(1980, 2024):
    url = f'https://www.go4go.net/go/games/finals/{i}'
    page = requests.get(url)
    raw_data = page.text
    input_data = raw_data.replace('<td><a href="https://www.go4go.net/go/games/sgfview/', 
                                  '<td>http://www.go4go.net/go/games/sgfview/')
    
    df_tournaments_temp = pd.read_html(input_data)[1]    
    frames_tournaments.append(df_tournaments_temp)
       
df_tournaments = pd.concat(frames_tournaments).reset_index()

df_tournaments['Result'] = df_tournaments['Result'].apply(lambda x: x.split(sep='"')[0])
df_tournaments['Tournament'] = df_tournaments['Tournament'].\
                                   apply(lambda x: x.split(sep=',')[0])
df_tournaments['id_game'] = df_tournaments['Result'].apply(lambda x: x.split(sep='/')[-1])

In [35]:
df_tournaments['Tournament_name'] = df_tournaments['Tournament'].\
                                        apply(lambda x: x.split(' ', 1)[1] 
                                              if x[0] in '123456789' else x)

In [36]:
df_tournaments

,index,Date,Tournament,Black,White,Result,id_game,Tournament_name
0,0,[1980-12-08],6th Japanese Tengen,Kato Masao,Yamabe Toshiro,http://www.go4go.net/go/games/sgfview/11002,11002,Japanese Tengen
1,1,[1980-11-26],6th Japanese Tengen,Yamabe Toshiro,Kato Masao,http://www.go4go.net/go/games/sgfview/11004,11004,Japanese Tengen
2,2,[1980-11-19],28th Japanese Oza,Kato Masao,Ishida Yoshio,http://www.go4go.net/go/games/sgfview/29806,29806,Japanese Oza
3,3,[1980-11-19],24th Kansai-Kiin 1st Place,Ishii Shinzo,Hashimoto Utaro,http://www.go4go.net/go/games/sgfview/62368,62368,Kansai-Kiin 1st Place
4,4,[1980-11-12],6th Japanese Tengen,Kato Masao,Yamabe Toshiro,http://www.go4go.net/go/games/sgfview/11003,11003,Japanese Tengen
...,...,...,...,...,...,...,...,...
5993,138,[2023-01-19],26th Japanese Female Kisei,Nakamura Sumire,Ueno Asami,http://www.go4go.net/go/games/sgfview/106931,106931,Japanese Female Kisei
5994,139,[2023-01-12],47th Japanese Kisei,Shibano Toramaru,Ichiriki Ryo,http://www.go4go.net/go/games/sgfview/106802,106802,Japanese Kisei
5995,140,[2023-01-07],2nd Korean Strongest Lady's Tournament,Choi Jeong,Kim Chaeyoung,http://www.go4go.net/go/games/sgfview/106611,106611,Korean Strongest Lady's Tournament
5996,141,[2023-01-06],2nd Korean Strongest Lady's Tournament,Kim Chaeyoung,Choi Jeong,http://www.go4go.net/go/games/sgfview/106610,106610,Korean Strongest Lady's Tournament


In [37]:
print(f"Общее число турниров: {len(df_tournaments['Tournament_name'].unique())}")
print(f"Общее число турниров с разделением по годам: {len(df_tournaments['Tournament'].unique())}")

Общее число турниров: 293
Общее число турниров с разделением по годам: 2294


In [38]:
df_parties = pd.read_csv('/kaggle/input/knowledge-graph-go/df_parties.csv')

In [39]:
df_tournaments['id_game'] = df_tournaments['id_game'].astype(int)
df_parties_tournaments = df_parties.merge(df_tournaments[['Tournament', 'Tournament_name', 'id_game']], 
                                          on='id_game', how='left')
df_parties_tournaments = df_parties_tournaments.fillna('-')

In [40]:
df_parties_tournaments

,Name,Date,Rating,Color,Result,Opponent,Opponent_Rating,Opponent_Sex,Opponent_Flag,Kifu,id_game,Tournament,Tournament_name
0,Ando Takeo,1976-03-03,2987,White,Loss,Miyazawa Goro,3075,man,jp,http://www.go4go.net/go/games/sgfview/80700,80700,-,-
1,Ando Takeo,1976-01-14,2988,White,Win,Kodama Kunio,2813,man,jp,http://www.go4go.net/go/games/sgfview/80685,80685,-,-
2,Li Chunhua,2016-09-22,2823,Black,Loss,Zhao Yifang,2961,woman,cn,http://www.go4go.net/go/games/sgfview/57892,57892,-,-
3,Li Chunhua,2016-09-13,2823,Black,Loss,Pan Yang,3012,woman,cn,http://www.go4go.net/go/games/sgfview/57320,57320,-,-
4,Li Chunhua,2016-09-03,2824,White,Loss,Zhang Zihan,3041,woman,cn,http://www.go4go.net/go/games/sgfview/56956,56956,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...
218686,Lee Seoyoung,2023-09-03,2811,Black,Win,Kim Sangin,2876,woman,kr,http://www.go4go.net/go/games/sgfview/110907,110907,-,-
218687,Hamakado Hibiki,2023-09-18,2894,White,Loss,Kuwabara Ki,2983,man,jp,http://www.go4go.net/go/games/sgfview/111207,111207,-,-
218688,Huang Qiuxuan,2023-09-24,2970,White,Loss,Hu Zihao,3361,man,cn,http://www.go4go.net/go/games/sgfview/111290,111290,-,-
218689,Kim Minji,2023-10-16,3041,Black,Loss,Choi Jeong,3430,woman,kr,http://www.go4go.net/go/games/sgfview/111649,111649,-,-


In [41]:
df_parties_tournaments[df_parties_tournaments['Tournament'] != '-']

,Name,Date,Rating,Color,Result,Opponent,Opponent_Rating,Opponent_Sex,Opponent_Flag,Kifu,id_game,Tournament,Tournament_name
27,Li Chunhua,2005-06-30,3020,Black,Loss,Ye Gui,3088,woman,cn,http://www.go4go.net/go/games/sgfview/6323,6323,3rd Chinese Female Mingren,Chinese Female Mingren
28,Li Chunhua,2005-06-29,3020,White,Loss,Ye Gui,3088,woman,cn,http://www.go4go.net/go/games/sgfview/6322,6322,3rd Chinese Female Mingren,Chinese Female Mingren
134,Li Daichun,2002-01-18,3279,Black,Win,Fu Li,3060,man,cn,http://www.go4go.net/go/games/sgfview/123,123,15th Chinese Wanbao Cup Amateur,Chinese Wanbao Cup Amateur
140,Li Daichun,2001-01-11,3280,Black,Win,Fu Li,3049,man,cn,http://www.go4go.net/go/games/sgfview/1121,1121,14th Chinese Wanbao Cup Amateur,Chinese Wanbao Cup Amateur
200,Lee Sedol,2018-05-18,3492,Black,Loss,Shin Jinseo,3700,man,kr,http://www.go4go.net/go/games/sgfview/74334,74334,23rd Korean GS Caltex Cup,Korean GS Caltex Cup
...,...,...,...,...,...,...,...,...,...,...,...,...,...
217945,Yoo Ohseong,2022-05-09,3300,Black,Loss,Park Yeonghun,3444,man,kr,http://www.go4go.net/go/games/sgfview/102995,102995,Year 2021 Korean League,Year 2021 Korean League
217946,Yoo Ohseong,2022-05-07,3300,White,Loss,Park Jonghoon,3396,man,kr,http://www.go4go.net/go/games/sgfview/102974,102974,Year 2021 Korean League,Year 2021 Korean League
218113,Park Sinyoung,2022-03-26,3169,Black,Win,Han Woojin,3345,man,kr,http://www.go4go.net/go/games/sgfview/102268,102268,7th Korean Future Stars Tournament,Korean Future Stars Tournament
218286,Won Jehun,2023-09-16,3121,White,Loss,Park Jihyun,3258,man,kr,http://www.go4go.net/go/games/sgfview/111159,111159,11th Hapcheon County Elite Tournament,Hapcheon County Elite Tournament


In [42]:
df_parties_tournaments.to_csv("df_parties_tournaments.csv", index=False)


## Winners and runners-up of tournaments

In [47]:
df_tournaments = df_parties_tournaments[df_parties_tournaments['Tournament'] != '-']
df_tournaments = df_tournaments[df_tournaments['Result'] == 'Win']

In [48]:
df_tournaments

,Name,Date,Rating,Color,Result,Opponent,Opponent_Rating,Opponent_Sex,Opponent_Flag,Kifu,id_game,Tournament,Tournament_name
134,Li Daichun,2002-01-18,3279,Black,Win,Fu Li,3060,man,cn,http://www.go4go.net/go/games/sgfview/123,123,15th Chinese Wanbao Cup Amateur,Chinese Wanbao Cup Amateur
140,Li Daichun,2001-01-11,3280,Black,Win,Fu Li,3049,man,cn,http://www.go4go.net/go/games/sgfview/1121,1121,14th Chinese Wanbao Cup Amateur,Chinese Wanbao Cup Amateur
201,Lee Sedol,2018-05-17,3492,White,Win,Shin Jinseo,3699,man,kr,http://www.go4go.net/go/games/sgfview/74302,74302,23rd Korean GS Caltex Cup,Korean GS Caltex Cup
203,Lee Sedol,2018-05-15,3493,White,Win,Shin Jinseo,3699,man,kr,http://www.go4go.net/go/games/sgfview/74254,74254,23rd Korean GS Caltex Cup,Korean GS Caltex Cup
231,Lee Sedol,2018-01-10,3518,Black,Win,Lian Xiao,3614,man,cn,http://www.go4go.net/go/games/sgfview/70630,70630,5th World Meijin Tournament,World Meijin Tournament
...,...,...,...,...,...,...,...,...,...,...,...,...,...
217326,Kim Hyoyoung,2022-02-04,3048,Black,Win,Kim Minseo,3021,woman,kr,http://www.go4go.net/go/games/sgfview/101272,101272,1st Korean Mediheal Cup Female Strongest,Korean Mediheal Cup Female Strongest
217385,Kim Beomseo,2022-07-23,3385,White,Win,Kwon Hyojin (m),3194,man,kr,http://www.go4go.net/go/games/sgfview/103922,103922,10th Hapcheon County Elite Tournament,Hapcheon County Elite Tournament
217386,Kim Beomseo,2022-07-20,3386,Black,Win,Kwon Hyojin (m),3194,man,kr,http://www.go4go.net/go/games/sgfview/103896,103896,10th Hapcheon County Elite Tournament,Hapcheon County Elite Tournament
217390,Kim Beomseo,2021-09-08,3383,White,Win,Hyun Yoobin,3224,man,kr,http://www.go4go.net/go/games/sgfview/98541,98541,2nd Korean New Star Choegowi,Korean New Star Choegowi


In [66]:
df_winners = df_tournaments.groupby(by=["Tournament"]).agg({'Date': "max"}).reset_index()

In [67]:
df_winners

,Tournament,Date
0,10th Asian TV Cup,1998-08-15
1,10th Chinese Agon Cup,2008-10-29
2,10th Chinese CCTV Cup,1998-05-22
3,10th Chinese Changqi Cup,2013-10-24
4,10th Chinese Jianqiao Cup,2012-11-13
...,...,...
2253,Year 2021 Chinese CCTV Cup,2021-12-05
2254,Year 2021 Korean League,2022-05-12
2255,Year 2022 Chinese CCTV Cup,2022-09-16
2256,Year 2022 Korean League,2023-06-25


In [68]:
df_winners = df_winners.merge(df_tournaments[['Date', 'Tournament', 'Tournament_name', 
                                              'Name', 'Opponent', 'Result']],
                              how='left',
                              left_on=['Date', 'Tournament'],
                              right_on=['Date', 'Tournament'])

df_winners = df_winners.drop(['Result'], axis=1)
df_winners.rename(columns = {'Name': 'Winner', 'Opponent': 'Runner-up'}, inplace = True)

df_winners

,Tournament,Date,Tournament_name,Winner,Runner-up
0,10th Asian TV Cup,1998-08-15,Asian TV Cup,Yoda Norimoto,Ma Xiaochun
1,10th Chinese Agon Cup,2008-10-29,Chinese Agon Cup,Gu Li,Chang Hao
2,10th Chinese CCTV Cup,1998-05-22,Chinese CCTV Cup,Cao Dayuan,Ma Xiaochun
3,10th Chinese Changqi Cup,2013-10-24,Chinese Changqi Cup,Shi Yue,Lian Xiao
4,10th Chinese Jianqiao Cup,2012-11-13,Chinese Jianqiao Cup,Wang Chenxing,Rui Naiwei
...,...,...,...,...,...
2354,Year 2022 Korean League,2023-06-25,Year 2022 Korean League,Park Jinsol,Hong Seongji
2355,Year 2022 Korean League,2023-06-25,Year 2022 Korean League,Kim Seongjae,Kwon Hyojin (m)
2356,Year 2022 Korean League,2023-06-25,Year 2022 Korean League,Kim Junghyun,Kim Changhoon
2357,Year 2022 Korean League,2023-06-25,Year 2022 Korean League,Shin Jinseo,Byun Sangil


In [69]:
df_winners[df_winners['Tournament'].str.contains("Korean Kuksu")]

,Tournament,Date,Tournament_name,Winner,Runner-up
892,26th Korean Kuksu,1982-12-10,Korean Kuksu,Cho Hunhyun,Seo Bongsoo
922,27th Korean Kuksu,1984-01-20,Korean Kuksu,Cho Hunhyun,Seo Bongsoo
952,28th Korean Kuksu,1985-03-04,Korean Kuksu,Cho Hunhyun,Seo Bongsoo
979,29th Korean Kuksu,1986-04-08,Korean Kuksu,Cho Hunhyun,Seo Bongsoo
1153,30th Korean Kuksu,1987-03-06,Korean Kuksu,Seo Bongsoo,Cho Hunhyun
1180,31st Korean Kuksu,1988-01-18,Korean Kuksu,Seo Bongsoo,Cho Hunhyun
1204,32nd Korean Kuksu,1988-12-28,Korean Kuksu,Cho Hunhyun,Seo Bongsoo
1225,33rd Korean Kuksu,1989-11-08,Korean Kuksu,Cho Hunhyun,Lee Changho
1246,34th Korean Kuksu,1990-10-10,Korean Kuksu,Lee Changho,Cho Hunhyun
1267,35th Korean Kuksu,1991-11-08,Korean Kuksu,Cho Hunhyun,Lee Changho


In [70]:
df_winners.to_csv("df_winners.csv", index=False)